In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from pandas.plotting import scatter_matrix

import statsmodels.formula.api as smf

pd.options.display.max_columns = 2000
pd.options.display.max_rows = 200

In [ ]:
def load_lending_club(path_):
    date_cols = ["issue_d", "earliest_cr_line", "last_pymnt_d", "last_credit_pull_d"]
    return pd.read_csv(path_, 
                 low_memory=False, 
                 parse_dates=date_cols, 
                 date_format="%b-%Y")

In [1]:
def tweak_lending_club(df_, ohe=False):
    drop_cols = ["debt_settlement_flag", "Unnamed: 0",  
                 "last_fico_range_low", "num_bc_sats", "num_bc_tl", 
                 "fico_range_low", "num_rev_accts", "out_prncp", 
                 "out_prncp_inv", "out_prncp_inv", "policy_code", 
                 "funded_amnt_inv", "loan_amnt", 
                 "collection_recovery_fee", "last_credit_pull_d", "id", 
                 "url", "pymnt_plan", "emp_title", 
                 "title", "zip_code", "verification_status", "last_pymnt_d"]
    high_nan_cols = ['all_util', 'annual_inc_joint', 'deferral_term', 
                     'dti_joint', 'hardship_amount', 'hardship_dpd', 'hardship_end_date', 
                     'hardship_last_payment_amount', 'hardship_length', 'hardship_loan_status', 
                     'hardship_payoff_balance_amount', 'hardship_reason', 'hardship_start_date', 
                     'hardship_status', 'hardship_type', 'il_util', 
                     'inq_fi', 'inq_last_12m', 'max_bal_bc', 
                     'mths_since_last_delinq', 'mths_since_last_major_derog', 'mths_since_last_record', 
                     'mths_since_rcnt_il', 'mths_since_recent_bc_dlq', 'mths_since_recent_revol_delinq', 
                     'next_pymnt_d', 'open_acc_6m', 'open_act_il', 
                     'open_il_12m', 'open_il_24m', 'open_rv_12m', 
                     'open_rv_24m', 'orig_projected_additional_accrued_interest', 'payment_plan_start_date', 
                     'revol_bal_joint', 'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med', 
                     'sec_app_earliest_cr_line', 'sec_app_fico_range_high', 'sec_app_fico_range_low', 
                     'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_num_rev_accts', 
                     'sec_app_open_acc', 'sec_app_open_act_il', 'sec_app_revol_util', 
                     'total_bal_il', 'total_cu_tl', 'verification_status_joint']
    transformed_cols = ["term", "initial_list_status", 
                        "application_type", "hardship_flag", "emp_length", 
                        "int_rate", "revol_util"]
    object_cols = ["grade", "sub_grade", "home_ownership", 
                   "purpose", "addr_state", "issue_date_month"]
    
    
    LC_df_ = df_.assign(target = np.where((df_.loan_status == "Charged Off") | 
                                        (df_.loan_status == "Does not meet the credit policy. Status:Charged Off") | 
                                        (df_.loan_status == "Default"), 
                                        1, 0), 
                      term_36months = np.where(df_.term == "60 months", 0, 1), 
                      initial_list_status_w = np.where(df_.initial_list_status == "f", 0, 1), 
                      individual = np.where(df_.application_type == "Joint App", 0, 1),
                      hardhship = np.where(df_.hardship_flag == "Y", 1, 0),
                      employed_over_10yrs = np.where(df_.emp_length == "10+ years", 1, 0),
                      emp_years = df_.emp_length.map({"< 1 year": 0, "1 year": 1, "2 years": 2, 
                                                        "3 years": 3, "4 years": 4, "5 years": 5, 
                                                        "6 years": 6, "7 years": 7, "8 years": 8, 
                                                        "9 years": 9, "10+ years": 10}),
                      interest_rate = df_.int_rate.str.replace('%', '').astype(float),
                      revolving_util = df_.revol_util.str.replace('%', '').astype(float),
                      issue_date_month = df_.issue_d.dt.month.astype(str),
                      issue_date_year = df_.issue_d.dt.year,
                      years_since_earliest_cr_line = df_.issue_d.dt.year - df_.earliest_cr_line.dt.year,
                     ).drop(labels=drop_cols+high_nan_cols+transformed_cols, axis=1)
        
    if ohe:
        return(pd.get_dummies(LC_df_, columns=object_cols).fillna(method="ffill"))
    else:
        return LC_df_
    